In [28]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import ParameterGrid

In [ ]:
## load data
final_result = pd.read_csv('/Users/pei/Desktop/Pei/Research/RA role in University/utd_prof_ding/processed_incident_count_005.csv')

# Define the folder path containing the CSV files
folder_path = '/Users/pei/Desktop/Pei/Research/RA role in University/utd_prof_ding/zDPD_data_005_all'  # Change this to your folder path

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

df = pd.DataFrame()

# Load each CSV file into a DataFrame and append to the list
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.concat([df, pd.read_csv(file_path).iloc[:,1:]])

# Display combined DataFrame (optional)
print(df.head())

   lon_bin  lat_bin        date  unique_event_count  date_year  tavg  tmin  \
0   -96.45    32.45  2024-04-25                   0       2024  22.2  19.0   
1   -96.45    32.45  2024-04-26                   0       2024  21.6  18.3   
2   -96.45    32.45  2024-04-27                   0       2024  24.2  19.1   
3   -96.45    32.45  2024-04-28                   0       2024  21.0  16.7   
4   -96.45    32.45  2024-04-29                   0       2024  20.7  14.4   

   tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun  \
0  24.8   1.5   NaN  151.0  13.1   NaN  1014.4   NaN   
1  23.5  37.1   NaN  163.0  18.0   NaN  1008.3   NaN   
2  28.3  36.1   NaN  150.0  22.1   NaN  1009.0   NaN   
3  24.1  65.0   NaN  156.0  16.1   NaN  1010.2   NaN   
4  27.5   0.0   NaN   18.0   2.4   NaN  1012.3   NaN   

                                    Statistical_Area  Unemployment_Rate(%)  \
0  Dallas-Fort Worth-Arlington, TX Metropolitan S...                   3.9   
1  Dallas-Fort Worth-Arlington, TX Met

In [6]:
df['lon_bin'] = df['lon_bin'].round(2)
df['lat_bin'] = df['lat_bin'].round(2)
df = df.merge(final_result, how = "left", 
              left_on = ['lon_bin', 'lat_bin', 'date'], right_on = ['lon_bin', 'lat_bin', 'date'],
              suffixes=['_drop', ''])
df.head(1)

,lon_bin,lat_bin,date,unique_event_count_drop,date_year,tavg,tmin,tmax,prcp,snow,...,wspd,wpgt,pres,tsun,Statistical_Area,Unemployment_Rate(%),Unemployment_Rank,CPI(Annual),Unnamed: 0,unique_event_count
0,-96.45,32.45,2024-04-25,0,2024,22.2,19.0,24.8,1.5,NaN,...,13.1,NaN,1014.4,NaN,"Dallas-Fort Worth-Arlington, TX Metropolitan S...",3.9,226,0.0,210000,0


In [ ]:
# df.unique_event_count.unique()

array([ 0,  1, 11,  4,  7,  6,  5,  8, 10,  3,  9,  2, 12, 41, 31, 21, 24,
       27, 28, 42, 30, 23, 36, 26, 29, 38, 35, 25, 34, 19, 14, 39, 17, 33,
       32, 22, 37, 46, 20, 13, 15, 45, 49, 51, 40, 16, 44, 43, 48, 47, 52,
       50, 55, 92, 67, 68, 18, 75, 65])

In [8]:
import xgboost as xgb
# import pandas as pd
# import numpy as np
from sklearn.metrics import root_mean_squared_error

# Ensure the data is sorted by date
df = df.sort_values(by='date')
# Assuming df['date'] contains datetime objects
df['date'] = pd.to_datetime(df['date'])

# Extract year, day of year, week of year, and month-day components
df['date_year'] = df['date'].dt.year
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week

# Encode day_of_year cyclically
df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Encode week_of_year cyclically
df['week_of_year_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_of_year_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)

In [37]:
# Define features and target
features = ['lon_bin', 'lat_bin', 'date_year', 'day_of_year_sin', 'day_of_year_cos', 'week_of_year_sin', 'week_of_year_cos',
            'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'Unemployment_Rate(%)', 'Unemployment_Rank', 'CPI(Annual)']
target = 'unique_event_count'

# Train-test split
train_data = df[df['date'] < '2024-01-01']  # Train on data before 2023
valid_data = df[(df['date'] >= '2024-01-01') & (df['date'] < '2024-03-01')]  # First two months of 2024 for validation
test_data = df[df['date'] >= '2024-03-01']  # Test on data from 2023 onward

# # Convert training data to DMatrix for XGBoost
# dtrain = xgb.DMatrix(train_data[features], label=train_data[target])

# poisson_obj for event count prediction assuming the count follow poisson distribution
def poisson_obj(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.exp(preds)  # Ensure predictions are positive
    grad = preds - labels  # Gradient: difference between prediction and true count
    hess = preds           # Hessian: prediction values (positive)
    return grad, hess

# Function to perform rolling validation
def rolling_validation(train_data, validation_data, params, features, target, num_boost_round=100):
    rolling_train_data = train_data.copy()  # Start with training data
    validation_dates = sorted(validation_data['date'].unique())  # Unique validation dates
    errors = []  # To store RMSE for each validation step
    predictions = {}
    true_values = {}

    for validation_date in validation_dates:
        # Get validation data for the current date
        validation_day = validation_data[validation_data['date'] == validation_date]
        dtrain = xgb.DMatrix(rolling_train_data[features], label=rolling_train_data[target])
        dvalidation = xgb.DMatrix(validation_day[features], label=validation_day[target])
        
        # Train the model
        model = xgb.train(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            verbose_eval=False,
            obj=poisson_obj
        )
        
        # Predict for the current validation day
        y_validation = validation_day[target].values
        y_pred = model.predict(dvalidation)
        y_pred_exp = np.exp(y_pred)  # Back-transform predictions
        y_pred_rounded = np.clip(np.round(y_pred_exp).astype(int), 0, None)  # Ensure non-negative integers
        
        # Calculate RMSE for the current day
        rmse = np.sqrt(np.mean((y_validation - y_pred_rounded) ** 2))
        errors.append(rmse)
        
        # Store predictions and true values
        predictions[validation_date] = y_pred_rounded
        true_values[validation_date] = y_validation
        
        # Incrementally update the training data with the validation day's data
        rolling_train_data = pd.concat([rolling_train_data, validation_day])
    
    # Return average RMSE, predictions, and true values
    return np.mean(errors), predictions, true_values, model

# Suggested coarse parameter grid for initial experimentation
param_grid = {
    'max_depth': [3, 6, 12],  # Include deeper trees for initial exploration
    'eta': [0.01, 0.1, 0.3],  # Learning rate
    'scale_pos_weight': [1, 10, 50],  # Class imbalance
    'subsample': [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
    'min_child_weight': [1, 5, 10],  # Minimum sum of weights for child nodes
    'objective': ['reg:squarederror'],  # Regression task
    'eval_metric': ['rmse']  # Evaluation metric
}

# Perform grid search with rolling validation on validation data
best_rmse = float("inf")
best_params = None

for params in ParameterGrid(param_grid):
    avg_rmse, _, _ = rolling_validation(train_data, valid_data, params, features, target, num_boost_round=100)
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_params = params

print(f"Best Parameters: {best_params}")
print(f"Best Validation RMSE: {best_rmse}")

# Hyperparameter grid search with rolling validation
param_grid = {
    'max_depth': [3, 6, 12],  # Controls the depth of the trees; coarser steps
    'eta': [0.01, 0.1, 0.3],  # Learning rate; coarser steps
    'scale_pos_weight': [1, 10, 50],  # To handle class imbalance, broader range
    'subsample': [0.6, 0.8, 1.0],  # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
    'min_child_weight': [1, 5, 10],  # Minimum sum of weights needed to create a child node
    'objective': ['reg:squarederror'],  # For regression tasks
    'eval_metric': ['rmse']  # Evaluation metric
}

best_rmse = float("inf")
best_params = None
params_search_record = {}
for params in ParameterGrid(param_grid):
    # Perform rolling validation for the current parameter combination
    avg_rmse, _, _ = rolling_validation(train_data, valid_data, params, features, target, num_boost_round=100)
    params_search_record[tuple(params)] = avg_rmse
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_params = params

print(f"Best Parameters: {best_params}")
print(f"Best Validation RMSE: {best_rmse}")

# Combine train and validation data for final testing
combined_train_valid = pd.concat([train_data, valid_data])

# Use the rolling_validation function on the test set
test_rmse, test_predictions, test_true_values, latest_model  = rolling_validation(
    combined_train_valid, test_data, best_params, features, target, num_boost_round=100
)

print(f"Test RMSE: {test_rmse}")

# # Train initial model
# combined_train_valid = pd.concat([train_data, valid_data])
# dtrain = xgb.DMatrix(combined_train_valid[features], label=combined_train_valid[target])
# final_model = xgb.train(best_params, dtrain, num_boost_round=100, obj=poisson_obj)


# # Rolling evaluation on the test set with one-step forecasting
# rolling_train_data = pd.concat([train_data, valid_data])  # Combine train and validation data
# test_dates = sorted(test_data['date'].unique())  # Get unique test dates
# errors = []  # To store RMSE for each test day
# predictions = {}
# true_values = {}

# # Train the initial model with the best parameters
# dtrain = xgb.DMatrix(rolling_train_data[features], label=rolling_train_data[target])
# model = xgb.train(best_params, dtrain, num_boost_round=100, obj=poisson_obj)

# for test_date in test_dates:
#     # Get test data for the current date
#     test_day = test_data[test_data['date'] == test_date]
#     dtest = xgb.DMatrix(test_day[features])
#     y_test = test_day[target].values  # True values for the current day
    
#     # Predict for the test day's data
#     y_pred = model.predict(dtest)
#     y_pred_exp = np.exp(y_pred)  # Back-transform predictions
#     y_pred_rounded = np.clip(np.round(y_pred_exp).astype(int), 0, None)  # Ensure non-negative integers
    
#     # Calculate RMSE for the current day
#     rmse = np.sqrt(np.mean((y_test - y_pred_rounded) ** 2))
#     errors.append(rmse)
    
#     # Store predictions and true values
#     predictions[test_date.strftime('%Y-%m-%d')] = y_pred_rounded
#     true_values[test_date.strftime('%Y-%m-%d')] = y_test
    
#     # Incrementally update the model with the test day's data
#     rolling_train_data = pd.concat([rolling_train_data, test_day])
#     dtrain = xgb.DMatrix(rolling_train_data[features], label=rolling_train_data[target])
#     model = xgb.train(best_params, dtrain, num_boost_round=100, obj=poisson_obj)

# # Overall evaluation
# overall_rmse = np.mean(errors)
# print(f"Overall RMSE on Test Data: {overall_rmse}")

Test RMSE: 0.9202952420031484


In [38]:
latest_model.save_model('model_20240429.xgb')

/opt/anaconda3/envs/crime_prediction/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [12:18:48] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [31]:
from collections import Counter

counter = Counter()
for arr in test_true_values.values():
    element_counts = Counter(arr)
    for key in element_counts.keys():
        counter[key] = counter.get(key, 0) + element_counts[key]
counter

Counter({0: 28206,
         1: 497,
         2: 329,
         3: 214,
         4: 192,
         5: 176,
         6: 168,
         7: 130,
         8: 128,
         9: 106,
         10: 78,
         11: 76,
         12: 65,
         13: 61,
         14: 46,
         16: 28,
         17: 26,
         18: 25,
         15: 23,
         20: 22,
         21: 18,
         19: 16,
         26: 13,
         22: 12,
         24: 7,
         25: 7,
         29: 6,
         23: 6,
         28: 5,
         31: 4,
         27: 3,
         30: 3,
         37: 3,
         39: 3,
         32: 3,
         42: 2,
         41: 2,
         33: 2,
         34: 2,
         35: 2,
         36: 1,
         40: 1,
         45: 1,
         75: 1,
         44: 1})

In [32]:
counter = Counter()
for arr in test_predictions.values():
    element_counts = Counter(arr)
    for key in element_counts.keys():
        counter[key] = counter.get(key, 0) + element_counts[key]
counter

Counter({0: 27886,
         1: 700,
         2: 316,
         7: 305,
         3: 266,
         8: 238,
         6: 172,
         4: 150,
         5: 104,
         9: 99,
         10: 60,
         15: 52,
         14: 47,
         19: 40,
         12: 40,
         16: 40,
         18: 27,
         17: 26,
         11: 26,
         20: 24,
         13: 23,
         29: 12,
         27: 9,
         31: 8,
         22: 8,
         21: 7,
         32: 7,
         30: 7,
         23: 5,
         26: 4,
         28: 3,
         24: 3,
         25: 2,
         34: 1,
         36: 1,
         40: 1,
         39: 1})

In [34]:
from sklearn.metrics import mean_squared_error
res = []
for key in test_predictions.keys():
    y_pred = test_predictions[key]
    y_true = test_true_values[key]

    ## mask out the zero values
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    res.append(np.sqrt(mean_squared_error(y_true_filtered, y_pred_filtered)))
print(np.mean(res))

3.111905793186735


In [35]:
from sklearn.metrics import mean_squared_error
res = []
for key in test_predictions.keys():
    y_pred = test_predictions[key]
    y_true = test_true_values[key]

    ## mask out the zero values
    mask = y_true == 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    res.append(np.sqrt(mean_squared_error(y_true_filtered, y_pred_filtered)))
print(np.mean(res))

0.22124508936260653


## Run from here ##

In [ ]:
import numpy as np
import pandas as pd

import xgboost as xgb

In [41]:
## Load the model
features = ['lon_bin', 'lat_bin', 'date_year', 'day_of_year_sin', 'day_of_year_cos', 'week_of_year_sin', 'week_of_year_cos',
            'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'Unemployment_Rate(%)', 'Unemployment_Rank', 'CPI(Annual)']
target = 'unique_event_count'

loaded_model = xgb.Booster()
loaded_model.load_model('model_20240429.xgb')

validation_day = df[df['date'] == '2024-04-29']
new_data = xgb.DMatrix(validation_day[features])

# Prepare new data for prediction
# new_data = xgb.DMatrix(new_features)  # `new_features` should be a DataFrame with the same features as the training set
new_predictions = loaded_model.predict(new_data)
new_predictions_exp = np.exp(new_predictions)  # Back-transform if using exponential in the objective

# Optionally round predictions if needed
new_predictions_rounded = np.clip(np.round(new_predictions_exp).astype(int), 0, None)

In [ ]:
new_predictions_rounded

512